In [13]:
import geopandas as gpd
from geopandas.geoseries import *

# 447 PLRs
file_path_of_plr = 'berlin-lor.planungsraeume.geojson'
plr = gpd.read_file(file_path_of_plr)
projection = "EPSG:5678" # https://epsg.io/5243 , https://epsg.io/5678 , https://epsg.io/3068
bufferSize = 300 # in meter
plr = plr.to_crs(projection)
 
# calc area (for fun ;-)) 
plr['area'] = None
plr['bufferedGeometry'] = None
plr['bufferedArea'] = None

plr['bufferedGeometry'] = plr.geometry.buffer(bufferSize)
plr['bufferedArea'] = None

for index, row in plr.iterrows():
    plr.loc[index, 'area'] = row['geometry'].area / 10000
    plr.loc[index, 'bufferedArea'] = row['bufferedGeometry'].area / 10000

# add open space to calc which of them intersects with bufferedGeometry 
file_path_of_open_spaces = 'gruenflaechen_shapes.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)
# 2509 open spaces
open_spaces = open_spaces.to_crs(projection)

open_spaces.head()

plr['openSpacesInBufferedPLR'] = np.empty((len(plr), 0)).tolist()
plr['openSpacesInPLR'] = np.empty((len(plr), 0)).tolist()

for indexPLR, rowPLR in plr.iterrows():
    for indexOS, rowOS in open_spaces.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(rowOS.KENNZEICH)
        if rowPLR.geometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(rowOS.KENNZEICH)

newplr = plr
newplr.astype({'geometry': str}, {'bufferedGeometry': str}).to_csv('PLR_buffered.csv', sep=';')


In [12]:


newplr['numberOfSpacesInPLR'] = None
newplr['numberOfSpacesInBufferedPLR'] = None

for index, row in newplr.iterrows():
    newplr.loc[index, 'numberOfSpacesInPLR'] = len(row.openSpacesInPLR)
    newplr.loc[index, 'numberOfSpacesInBufferedPLR'] = len(row.openSpacesInBufferedPLR)


newplr.head()

# TODO
# fläche der Gründflächen (buffered and non) pro PLR
# Menschen pro PLR
# Als Einwohner von PLR x hat man y Grünflächen mit z m^2 Fläche zur Verfügung
# Daten für Waldflächen

,SCHLUESSEL,PLR_NAME,geometry,area,bufferedGeometry,bufferedArea,openSpacesInBufferedPLR,openSpacesInPLR,numberOfSpacesInPLR,numberOfSpacesInBufferedPLR
0,01011101,Stülerstr.,"POLYGON ((4591014.544 5819985.375, 4591080.254...",36.7055,"POLYGON ((4590729.984 5819890.373, 4590722.091...",150.52,"[216210, 400000, 217100, 242700, 216810, 21670...","[400000, 242700, 212610]",3,7
1,01011102,Großer Tiergarten,"POLYGON ((4590490.781 5820805.462, 4590491.763...",301.156,"POLYGON ((4590161.505 5820667.145, 4590158.549...",608.388,"[253500, 244010, 253400, 161900, 210100, 21269...","[253500, 244010, 210100, 212690, 400000, 24270...",21,50
2,01011103,Lützowstr.,"POLYGON ((4591721.796 5819736.719, 4591754.196...",52.2729,"POLYGON ((4591533.106 5819979.445, 4591539.149...",176.848,"[216310, 216500, 216210, 206110, 216400, 21710...","[216500, 216210, 216400]",3,9
3,01011104,Körnerstr.,"POLYGON ((4592632.388 5819374.203, 4592639.827...",33.8588,"POLYGON ((4592545.285 5819087.126, 4592517.674...",132.499,[],[],0,0
4,01011105,Nördl. Landwehrkanal,"POLYGON ((4591867.807 5820484.277, 4591895.354...",93.5224,"POLYGON ((4591590.290 5820370.327, 4591580.473...",272.056,[],[],0,0
